<h1> Introduction to LangGraph </h1>

LangGraph is a framework for creating applications using graph-based workflows. Each node represents a function or computational step, and edges define the flow between these nodes based on certain conditions.

<h2> Key Features: </h2>

State Management
Flexible Routing
Persistence
Visualization

Overview: Text Analysis Pipeline

In this, let see the power of LangGraph by building a multi-step text analysis pipeline. Our use case will focus on processing a given text through three key stages:

**Text Classification:** We'll categorize the input text into predefined categories (e.g., News, Blog, Research, or Other).
**Entity Extraction:** We'll identify and extract key entities such as persons, organizations, and locations from the text.
**Text Summarization:** Finally, we'll generate a concise summary of the input text.

This pipeline showcases how LangGraph can be used to create a modular, extensible workflow for natural language processing tasks. By the end of this tutorial, you'll understand how to construct a graph-based application that can be easily modified or expanded for various text analysis needs.

In [1]:
# importing libraries
import os
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

<h2> Building the Text Processing Pipeline </h2>

<h3> Define State and Initialize LLM </h3>

Here we define the State class to hold our workflow data and initialize the ChatOpenAI model.

In [2]:
class State(TypedDict):
    text : str
    classification : str
    entities : List[str]
    summary : str

In [3]:
# Initialize LLM
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)

## Define Node Functions

These functions define the operations performed at each node of our graph: classification, entity extraction and summarization

In [4]:
def classification_node(state : State):
    '''
    Classify the text into one of the categories  : News, Blog, Research, Other
    '''
    prompt = PromptTemplate(
        input_variables = ["text"],
        template = "Classify the following text into one of the categories: News, Blog, Research, or Other.\n\nText:{text}\n\nCategory:"        
    )
    message = HumanMessage(content=prompt.format(text=state["text"]))
    classification = llm.predict_messages([message]).content.strip()

    return {"classification":classification}

In [5]:
def entity_extraction_node(state: State):
    ''' Extract all the entities (Person, Organization, Location) from the text '''
    prompt = PromptTemplate(
        input_variables=["text"],
        template="Extract all the entities (Person, Organization, Location) from the following text. Provide the result as a comma-separated list.\n\nText:{text}\n\nEntities:"
    )
    message = HumanMessage(content=prompt.format(text=state["text"]))
    entities = llm.predict_messages([message]).content.strip().split(", ")
    return {"entities": entities}

In [ ]:
def summarization_node(state: State):
    ''' Summarize the text in one short sentence '''
    prompt = PromptTemplate(
        input_variables=["text"],
        template="Summarize the following text in one short sentence.\n\nText:{text}\n\nSummary:"
    )
    message = HumanMessage(content=prompt.format(text=state["text"]))
    summary = llm.predict_messages([message]).content.strip()
    return {"summary": summary}